In [1]:
from IPython.display import HTML
from IPython.display import display

# Taken from https://stackoverflow.com/questions/31517194/how-to-hide-one-specific-cell-input-or-output-in-ipython-notebook
tag = HTML('''<script>
code_show=true; 
function code_toggle() {
    if (code_show){
        $('div.cell.code_cell.rendered.selected div.input').hide();
    } else {
        $('div.cell.code_cell.rendered.selected div.input').show();
    }
    code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
Some hidden code... to show/hide this cell's raw code input, click (or double click if stubborn!) <a href="javascript:code_toggle()">here</a>.''')
display(tag)

############### Write code below ##################
from IPython.core.display import HTML
def css_styling():
    styles = open("./STUFF/colors.css", "r").read()
    return HTML(styles)
css_styling()

# CH222

# Studying the solvation shell of ions via Molecular Dynamics Simulations

In this computational experiment you will use **molecular dynamics simulations** to investigate the solvation shell of ions at the atomistic level. For this computer experiment the ions and the solvent particles will be represented by relatively simple models (more on that later), and you will examine how the size of the cation and the temperature effect the structure of the solvation shell.

You will use the Python-based Jupyter Notebook framework, leveraging Statistical Mechanics to compute structural features of this system.

This notebook outlines how to set up the MD simulation using the HOOMD-blue simulation package, and some tips on using the Atomic Simulation Environment (ASE) for handling atomic configurations.

You can find further help, features and options on the Manual pages:  
[HOOMD-blue page](https://hoomd-blue.readthedocs.io/en/stable/index.html)  
[ASE page](https://wiki.fysik.dtu.dk/ase/index.html)

<div class=warn>
Tasks, where you need to edit the code cells will be shown in similar "blue boxes".
<div/>

**IMPORTANT!!!** Use **Google Chrome** to work on the notebook instead of using Microsoft Explorer. The latter handles the visualisation package incorrectly.

## Content of this notebook

#### Setting up the simulation (this Notebook)

1. Import necessary libraries and initialise HOOMD
2. Set simulation parameters
3. Create initial configuration
4. Initialise the MD engine
5. Set the output format - output file and logged quantities
6. Perform the MD simulation - hoomd.run()

#### Part 2 ("M15_Part.2_ANALYSIS.ipynb" Notebook) Analysing the results

7. Analysing results

## 1. Import necessary libraries and initialise HOOMD

In [ ]:
import hoomd
import hoomd.md
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
import math
from scipy.signal import find_peaks
import numpy as np
from matplotlib import pyplot 
import ase # Atomic simulation environment
import ase.io
from ase.visualize import view
#packages necessary to visualise the trajectories
import pytraj as pt 
import nglview as nv

hoomd.context.initialize('--mode=cpu')

def save_config(hoomd_system,atom_types={"Cation":"C","Solvent":"S","Anion":"Ar"}):
    
    lattice=np.array([hoomd_system.box.get_lattice_vector(i=i) for i in range(3)])
    
    pos=[system.particles[i].position for i in range(hoomd_system.particles.types.pdata.getN())]
    types=[atom_types[system.particles[i].type] for i in range(hoomd_system.particles.types.pdata.getN())]
    ase_atoms = ase.Atoms(pbc=(True,True,True),cell=lattice,positions=pos, symbols=types)
    
    ase_atoms.wrap()
    return ase_atoms

## 2. Set simulation parameters
Set the values for the size of the system, potential parameters, temperature, pressure...etc.
(A note about the units: since we use the Lennard-Jones potential, the sigma parameter controls the lengthscale of the simulation, the epsilon parameter the energy. Although we could simple work in these Lennard-Jones units, for the sake of easy interpretation we use units and values that feel natural in an atomic system.) 

<div class=warn>
Set the initial box length and the number of solvents below. Note: do not deviate too much from the original values, in order to avoid very high/low densities. </div>

In [ ]:
boxL=18.0 # Angstrom
N_solvent=85 # number of solvents per ion pair

temperature=    #0.8 # in units of kT
pressure=       #0.1 # in units of kJ/mol/Angstrom^3 which is approximatly 160 MPa 
timestep=0.5    # length of one timestep in units of femtosecond 

solvent_sigma=1.0 # Angstrom
solvent_epsilon=1.0 # kJ/mol

anion_sigma=    #1.0 # Angstrom
anion_epsilon=  #1.0 # kJ/mol

cation_sigma=   #2.0 # Angstrom
cation_epsilon= #1.0 # kJ/mol

## 3. Create initial configuration

First we need to create the **inital configuration** for the simulation to start from: 
We create an initial cubic cell with randomly positioned atoms, then replicate this cell in each Cartesian direction (3 times) to create our actual simulation box.

(Note: You can define the system only once, thus you can execute the code cell below only once. If you want to experiment with different setups, you need to restart the kernel or initialize the hoomd.context again!)

In [ ]:
position=[]
type_name=[]
charge=[]
mass=[]

#Add one cation
position.append([0.0, 0.0, 0.0])
type_name.append('Cation')
charge.append(1.0)
mass.append(1.0)
    
#Add one anion    
position.append([3.0, 3.0, 3.0])
type_name.append('Anion')
charge.append(-1.0)
mass.append(1.0)

boxL=boxL/3.0
#Add solvent particles
for i in range(N_solvent):
    #print("Adding atom", i, "of",N_solvent)
    generate_new=True
    while generate_new: 
        trial=np.random.rand(3)*(boxL-1.0)
        generate_new=False
        for j in range(i+2):
            if (np.linalg.norm(position[j]-trial)<1.0):
                generate_new=True
    position.append(trial)
    type_name.append('Solvent')
    charge.append(0.0)
    mass.append(1.0)
    
#Create the initial cell    
initial_cell = hoomd.lattice.unitcell(N = N_solvent+2, # number of atoms in the initial cell
                a1 = [boxL,0,0],
                a2 = [0,boxL,0],
                a3 = [0,0,boxL], # these define a cubic box of length boxL 
                position = position, # set positions for the two atoms (they should not be placed on top of each other)
                type_name = type_name,
                charge = charge,
                mass = mass,  
                            );

#Using the initial cell as a building block, create a larger cell. This one will be used for the simulation.
system=hoomd.init.create_lattice(initial_cell,n=3) 

<div class=warn>
Now it's time to have a look at our system - It's always a good idea to see what we have created!  </div>

We need a "topology file" (a single configuration of the system containing information about which and how many atoms we have, the simulation box, etc...) and a "trajectory file", which contains information about the time evolution of the system. 

HOOMD writes the trajectory file as a binary .dcd file. In order to write the topology file, we will make use of the Atomic Simulation Environment (ASE) instead. The short ``save_config`` function (included at the beginning of the notebook) extracts the simulation box parameters and the atomic coordinates from the hoomd ``system`` we have defined, and creates an atoms object from it. 

**Note:** ASE expects real atomic symbols as atom types, thus e.g. ``Solvent`` will not work. Hence, we will call our particle types S (solvent), C (Cation) and Ar (Anion), this will only effect the visualisation and how you refer to the atoms during the analysis code. 

In [ ]:
# Visualise the starting configuration
current_config = save_config(system)

# Writes a topology file in .pdb format via ASE 
ase.io.write("system.pdb", current_config)

# Use the py_traj package to load the topology file (we don't have trajectory file as we have not run any MD yet!)
conf_ini = pt.load('system.pdb')

# Show...
# NOTE: if you are running the entire Notebook from top to bottom in one go 
# by e.g. "Kernel -> Restart & Run all" you'll have to wait for the kernel to
# execute ALL cells before the visualisation shows up!
p_view = nv.show_pytraj(conf_ini)
p_view.add_unitcell()
p_view

## 4. Initialise the MD engine

Initialise the potential model, the way the neighbour list of particles are calculated and the how the long range electrostatic interaction is calculated.
We also initialise the thermostat and the barostat to keep the temperature and pressure constant. Define the timestep.

In [ ]:
nl = hoomd.md.nlist.cell() # neighbour list to keep track of the atomic distances - this makes the calculation more efficient
lj = hoomd.md.pair.lj(r_cut=3.5, nlist=nl) # define potential
lj.pair_coeff.set('Solvent', 'Solvent', epsilon=solvent_epsilon, sigma=solvent_sigma)
lj.pair_coeff.set('Solvent', 'Anion',  epsilon=np.sqrt(solvent_epsilon*anion_epsilon), sigma=(solvent_sigma+anion_sigma)/2.0)
lj.pair_coeff.set('Solvent', 'Cation', epsilon=np.sqrt(solvent_epsilon*cation_epsilon), sigma=(solvent_sigma+cation_sigma)/2.0)
lj.pair_coeff.set('Anion', 'Cation', epsilon=np.sqrt(cation_epsilon*anion_epsilon), sigma=(cation_sigma+anion_sigma)/2.0)
lj.pair_coeff.set('Anion', 'Anion', epsilon=anion_epsilon, sigma=anion_sigma)
lj.pair_coeff.set('Cation', 'Cation', epsilon=cation_epsilon, sigma=cation_sigma)
#define long range electrostatic interaction: using the particle-particle-particle mesh Ewald method.
charged = hoomd.group.charged()
all = hoomd.group.all()
pppm = hoomd.md.charge.pppm(group=charged, nlist=nl)
pppm.set_params(Nx=32, Ny=32, Nz=32, order=5, rcut=3.5)
# define timestep
hoomd.md.integrate.mode_standard(dt=timestep/100); # to convert it to units hoomd expects it
# initialise thermostat and barostat to be able to use different ensembles
nvt = hoomd.md.integrate.langevin(group=all, kT=temperature, seed=10)
npt = hoomd.md.integrate.npt(group=all, kT=temperature, tau=3.0, P=pressure, tauP = 3.0, couple="xyz")

## 5. Set the output format

What quantities should be collected, how often, and where they should be saved during the MD simulation?
A number of quantities are calculated during a simulation, and these can be logged so we can check and analyse them later. A few example of useful quantities are below, you can choose which ones to include (or all of them). The log file is a text file, with each column corresponding to the quantities defined, in the same order. Output data is written to the file line by line at every period timestep.

<div class=warn> Define the output filenames you would like to use. </div>
(The setting below will result in a filename reflecting the temperature and pressure of the simulation, but you are free to choose a different filename if you wish.)

In [ ]:
log_filename="MD_T{:.3f}_p{:.3f}.out".format(temperature,pressure)
dcd_filename="MD_T{:.3f}_p{:.3f}.dcd".format(temperature,pressure)
quantities=['volume','lx','ly','lz','potential_energy','kinetic_energy','temperature','pressure']

hoomd.analyze.log(filename=log_filename,
                  quantities=quantities,
                  period=50,
                  overwrite=True) # keep appending the existing file

d = hoomd.dump.dcd(dcd_filename, period=250, group=all, overwrite=True, unwrap_full=False);

# 6. All is set. We are ready to go!

Now all parameters are set, you can start the actual molecular dynamics simulation, hurray!
The initial configuration is usually very far from the equilibrium, and to lessen large initial fluctuations, it is usually a good idea to start with a short ($N$,$V$,$T$) simulation before switching to the ($N$,$P$,$T$) ensemble. 

Only one integrator can be set at a time, so just to be sure we'll be using the right thing, we disable the npt integrator and enable the nvt. (This might give a warning that nvt has been already enabled, that's okay.)

**Note**: hoomd will throw a warning about the fact that we are enabling the `nvt` framework whilst it has been enabled already. That's ok, we added that line (which is indeed redundant, as when we have defined both `npt` and `nvt` that enabled them both) to make clear that you have to disable/enable the framework of choice before running the MD. 

In [ ]:
npt.disable()
nvt.enable()
# Ignore the warning...
hoomd.run(2000) # perform 2000 MD steps.

``2000`` steps should take less than one minute. If you perform the ``hoomd.run()`` command again, the simulation will continue from the last configuration available.  

After the initial pre-equilibration of the system, on the ($N$,$V$,$T$) ensemble, we switch to the ($N$,$P$,$T$) ensemble to equilibrate the density of the system as well. Note, the ($N$,$P$,$T$) simulation is a bit slower, especially if the volume decreases (larger density means more neighbouring atoms, thus more pair-energies to compute!)

In [ ]:
nvt.disable()
npt.enable()

In [ ]:
hoomd.run(10000)

In [ ]:
# Visualise the entire trajectory (NVT + NPT!)
# If you want to split the two runs you will have to modify the output files above
# As at the moment we are "appending" our results in terms of observables (temperature, pressure...) 
# and trajectories to md.log and dump.dcd, respectively

# Load the trajectory and the topology
p_traj = pt.load(dcd_filename, top='system.pdb')
p_view = nv.show_pytraj(p_traj)
p_view

#### The visualisation is informative, but we need some more solid data to see whether the simulation is converged

Check the status of the simulation (e.g. how the potential energy, the volume, the temperature...etc. changes. Do they fluctuate widely or are converged around a certainn value?). Saved quantities can be read from the produced logfile, and plotted as required. Keep in mind that the order of the quantities is what you have defined at the point of creating the log, (the zeroth column is always the timstep).

<div class=warn> Create two graphs to check the evolution of the (a) volume of the simulation box and (b) the potential energy of the system. Remember to label the axis correctly (i.e. including units!). </div>

In [ ]:
data = np.genfromtxt(fname=log_filename, skip_header=True)
# Remember:
# quantities=['volume','lx','ly','lz','potential_energy','kinetic_energy','temperature','pressure']

In [ ]:
# pyplot.figure(figsize=(8,4), dpi=100)
fig=plt.figure(num=None,figsize=(5,3),dpi=200,facecolor='w',edgecolor='k')
pyplot.plot(data[:,0]*0.5,data[:,1]) # convert timestep to time
pyplot.xlabel('Time [fs]')
pyplot.ylabel('Volume (Å$^3$)')
plt.show()

How can you decide whether your run is equilibrated and is ready to start collecting data?  

If you still see a systematic change in the volume or the energy, thus not yet reached the phase where these quantities fluctuate around an average value, you need to continue the simulation a bit longer! Execute the command `hoomd.run(10000)` again until you're satisfied the system reached equilibrium. In a markdown cell, briefly discuss the outcome of your equilibration protocol with specific reference to the graphs you have generated. </div>